# 高阶函数

Haskell 中的函数可以接受函数作为参数，也可以将函数用作返回值，这样的函数被称作**高阶函数**。在函数式编程语言中，高阶函数不可或缺。

## 柯里函数

本质上，Haskell 的所有函数都只有一个参数，我们见过的所有多参数的函数都是柯里函数。**柯里函数**不会一次性取完所有参数，而是在每次调用时只取一个参数，并返回一个一元函数来取下一个参数，以此类推。

In [2]:
max 4 5
(max 4) 5
-- 这两个调用是等价的

Line 2: Redundant bracket
Found:
(max 4) 5
Why not:
max 4 5

5

5

柯里函数的好处是：我们只要以部分的参数来调用某函数，就可以得到一个**部分应用（partial application）函数**（或称**偏函数**），部分应用函数所接收的参数的数量，和之前少传入的参数的数量一致。

In [1]:
compareWithHundred :: Int -> Ordering
compareWithHundred x = compare 100 x
compareWithHundred'  = compare 100

compareWithHundred 99
compareWithHundred' 99
-- 由于柯里化，这两种定义是等价的
-- 通常，如果你的函数类似于 foo a = bar b a，就可以改为 foo = bar b，因为有柯里化。

Line 2: Eta reduce
Found:
compareWithHundred x = compare 100 x
Why not:
compareWithHundred = compare 100

GT

GT

通过**截断（section）**，我们也可以对中缀函数进行部分应用。将一个参数放在中缀函数的一侧，并在外面用括号括起，即可截断这个中缀函数。最终得到一个一元函数，其参数代表中缀函数剩余的参数。如下所示：

In [8]:
divideByTen :: (Floating a) => a -> a
divideByTen = (/10)

divideByTen 200
200 / 10
(/10) 200

20.0

20.0

20.0

使用截断时，唯一需要注意的是`-`（负号或者减号）运算符。按照截断的定义，`(-4)`理应返回一个使参数减去4的一元函数。然而为方便起见，Haskell中`(-4)`表示的是负4。因此，如果你需要一个将参数减4的函数，就部分应用`subtract`函数好了，像这样：`(subtract 4)`。

`zipWith`：取一个函数和两个列表作为参数，然后使用两个列表中相应的元素去调用该函数，将两个列表结合到一起。如下所示：

In [10]:
zipWith' :: (a -> b -> c) -> [a] -> [b] -> [c]
zipWith' _ [] _ = []
zipWith' _ _ [] = []
zipWith' f (x:xs) (y:ys) = f x y : zipWith' f xs ys

zipWith' (+) [4,2,5,6] [2,6,2,3]

[6,8,7,9]

`flip`：取一个函数作为参数，返回一个效果相同的新函数，两个函数的唯一区别是，新函数的前两个参数的顺序和原先的函数前两个参数的顺序正好颠倒。如下所示：

In [11]:
flip' :: (a -> b -> c) -> (b -> a -> c)
flip' f = g
    where g x y = f y x

-- flip' :: (a -> b -> c) -> b -> a -> c
-- flip' f y x = f x y
-- 这两种定义是等价的

`flip`处理过的函数一般都用来传给其他函数，如下所示：

In [13]:
zip [1,2,3,4,5] "hello"
flip' zip [1,2,3,4,5] "hello"

[(1,'h'),(2,'e'),(3,'l'),(4,'l'),(5,'o')]

[('h',1),('e',2),('l',3),('l',4),('o',5)]

## 常用高阶函数

### `map`

`map`取一个函数和一个列表作为参数，它会将这个函数应用到该列表的每个元素，产生一个新的列表。如下所示：

In [15]:
map' :: (a -> b) -> [a] -> [b]
map' _ [] = []
map' f (x:xs) = f x : map f xs

map (+3) [1,5,3,1,6]
map' (+3) [1,5,3,1,6]

[4,8,6,4,9]

[4,8,6,4,9]

### `filter`

`filter`函数取一个谓词（predicate）和一个列表，返回由列表中所有符合该条件的元素组成的列表。（**谓词**特指判断事物为`True`还是`False`的函数；也就是返回布尔值的函数）如下所示：

In [17]:
filter' :: (a -> Bool) -> [a] -> [a]
filter' _ [] = []
filter' p (x:xs)
    | p x       = x : filter p xs
    | otherwise = filter p xs

filter (>3) [1,5,3,2,1,6,4,3,2,1]
filter' (>3) [1,5,3,2,1,6,4,3,2,1]

[5,6,4]

[5,6,4]

## lambda

`lambda`就是一次性的匿名函数，有些时候我们需要传给高阶函数一个特定功能的函数，这就会用到`lambda`。要声明一个`lambda`，就写一个`\`（因为它看起来像是希腊字母的 —— 如果你斜着看），后跟函数的参数列表，参数之间用空格分隔，`->`后面是函数体。通常我们习惯将整个`lambda`用括号括起来。如下所示：

In [19]:
chain :: Integer -> [Integer]
chain 1 = [1]
chain n
    | even n = n:chain (n `div` 2)
    | odd n = n:chain (n*3 + 1)

numLongChains :: Int
numLongChains = length (filter (\xs -> length xs > 15) (map chain [1..100]))

numLongChains

66

和普通函数一样，`lambda`也可以取多个参数：

In [21]:
zipWith (\a b -> (a * 30 + 3) / b) [5,4,3,2,1] [1,2,3,4,5]

[153.0,61.5,31.0,15.75,6.6]

此外，同普通函数一样，你也可以在`lambda`中使用模式匹配，不过你无法为一个参数设置多个模式（如`[]`和`(x:xs)`，将后者作为前者的后备模式）。如下所示：

In [22]:
map (\(a,b) -> a + b) [(1,2), (3,5), (6,3), (2,6), (2,5)]

Line 1: Use uncurry
Found:
\ (a, b) -> a + b
Why not:
uncurry (+)

[3,8,9,8,7]

只要`lambda`的模式匹配失败，就会引发一个运行时错误，所以请慎用！

当编写不带括号的`lambda`时，Haskell会假定箭头`->`的右侧都是`lambda`的函数体。

## fold 函数

fold函数即折叠函数，折叠允许我们将一个数据结构（像列表）归约（reduce）为单个值。

所有遍历列表中元素并据此返回一个值的操作都可以基于折叠实现。无论何时需要遍历列表并返回某个值，都可以尝试下折叠。

一个折叠取一个**二元函数**（取两个参数的函数，如`+`和`div`）、一个初始值（通常称为**累加值**）以及一个待折叠的列表。

列表可以从左端开始折叠，也可以从右端开始折叠。折叠函数会取初始值和列表的起始元素来应用二元函数，得到返回值作为新的累加值。然后，以新的累加值和新的列表第一个元素（或最后一个元素）调用该函数，依次类推。到列表遍历完毕时，会只剩一个累加值，也就是最终的结果。

### `foldl`

`foldl`函数又被称为左折叠，它从列表的左端开始折叠，用初始值和列表的头部调用二元函数，得到一个新的累加值，并用新的累加值与列表的下一个元素调用二元函数，以此类推。以实现`sum`函数为例：

In [41]:
sum' :: (Num a) => [a] -> a
sum' xs = foldl (\acc x -> acc + x) 0 xs
sum'' = foldl (+) 0

sum' [3,5,2,1]
sum'' [3,5,2,1]
-- 这两种是等价的

Line 2: Eta reduce
Found:
sum' xs = foldl (\ acc x -> acc + x) 0 xs
Why not:
sum' = foldl (\ acc x -> acc + x) 0Line 2: Avoid lambda
Found:
\ acc x -> acc + x
Why not:
(+)Line 3: Use sum
Found:
foldl (+) 0
Why not:
sum

11

11

### `foldr`

右折叠函数`foldr`的行为与左折叠相似，不过累加值是从列表的右端开始。此外，右折叠的二元函数的参数顺序也是相反的：第一个参数是当前列表值，第二个参数是累加值。（将累加值放在右侧是合理的，毕竟是从右端开始折叠。）

折叠中的累加值（返回值也同样）可以是任何类型，可以是数值、布尔甚至一个新的列表。以实现`map`函数为例：

In [42]:
map' :: (a -> b) -> [a] -> [b]
map' f xs = foldr (\x acc -> f x : acc) [] xs  -- 通过右折叠实现
map'' f xs = foldl (\acc x -> acc ++ [f x]) [] xs  -- 通过左折叠实现

map' (+3) [1,2,3]

Line 2: Eta reduce
Found:
map' f xs = foldr (\ x acc -> f x : acc) [] xs
Why not:
map' f = foldr (\ x acc -> f x : acc) []Line 2: Use map
Found:
foldr (\ x acc -> f x : acc) []
Why not:
map (\ x -> f x)

[4,5,6]

上面两种实现方式，显然右折叠实现方式效率更高，因为使用（`++`）往列表后面追加元素的效率要比使用（`:`）低得多。所以需要生成新列表的时候，一般倾向于使用右折叠。

此外，两种折叠有一大区别：左折叠无法处理无限列表。而右折叠可以。

### `foldl1` 与 `foldr1`

`foldl1`与`foldr1`的行为与`foldl`和`foldr`相似，只是无需明确提供初始值。它们假定列表的第一个（或最后一个）元素是初始值，并从旁边的元素开始折叠。以实现`maximum`为例：

In [1]:
maximum' :: (Ord a) => [a] -> a
maximum' = foldl1 max

maximum' [3,6,4,1,9]

Line 2: Use maximum
Found:
foldl1 max
Why not:
maximum

9

这里要求待折叠的列表中至少含有一个元素，若传入空列表就会产生一个运行时错误。不过使用`foldl`和`foldr`处理空列表就不会遇到问题。

在处理折叠时，应先想一下它在遇到空列表时会怎样。如果该函数在理论上不应该处理空列表，可以考虑使用`foldr1`和`foldl1`。

### `scanl`、`scanr`、`scanl1`、`scanr1`

`scanl`和`scanr`两个函数与`foldl`和`foldr`相似，不过它们会将累加值的所有变动记录到一个列表中。也有`scanl1`和`scanr1`，它们相当于`foldl1`和`foldr1`。

## 函数应用符

`$`函数也叫做**函数应用符**（function application operator），普通的函数应用（通过空格隔开）有最高的优先级，而`$`的优先级则最低。用空格的函数调用符是左结合的（如`f a b c`与`((f a) b) c`等价），而`$`则是右结合的。

在大多数情况下，它可以减少代码中括号的数目。

你可以将`$`看作是在表达式右侧写一对括号的等价形式。如下所示：

In [26]:
sum (filter (>10) (map (*2) [2..10]))

sum $ filter (>10) $ map (*2) [2..10]

-- 这两种是等价的

80

80

除减少括号外，`$`还可以将函数应用转为函数，这就允许我们映射一个函数应用到一组函数组成的列表：

In [27]:
map ($ 3) [(4+), (10*), (^2), sqrt]

[7.0,30.0,9.0,1.7320508075688772]

## 函数组合

**函数组合**即组合两个函数，这就相当于先拿参数调用一个函数，然后取结果作为参数调用下一个函数。进行函数组合的函数为`.`，如下所示：

In [29]:
map (\xs -> negate (sum (tail xs))) [[1..5], [3..6], [1..7]]

map (negate . sum . tail) [[1..5], [3..6], [1..7]]

-- 这两种是等价的

Line 1: Avoid lambda
Found:
\ xs -> negate (sum (tail xs))
Why not:
negate . sum . tail

[-14,-15,-27]

[-14,-15,-27]

对于带有多个参数的函数的组合，通过部分应用使每个函数都只剩下一个参数就好了，如下所示：

In [33]:
sum (replicate 5 (max 6.7 8.9))

(sum . replicate 5) (max 6.7 8.9)

sum . replicate 5 $ max 6.7 8.9

-- 这三种是等价的

44.5

44.5

44.5

如果你打算用函数组合来省去表达式中大量的括号，可以先找出最里面的函数及参数写下来，在它们前面加一个`$`，接着略去其余函数的最后一个参数，通过`.`组合到一起。如下所示：

In [36]:
replicate 2 (product (map (*3) (zipWith max [1,2] [4,5])))
replicate 2 . product . map (*3) $ zipWith max [1,2] [4,5]
-- 这两种是等价的

[180,180]

[180,180]

如果遇到表达式以三个括号结尾，使用函数组合的方式进行重写之后，会用到两次函数组合符。